# 使用高速过滤器标注全部数据

In [1]:
#!AHOCORASICK_BYTES=1 pip install git+https://github.com/WojciechMula/pyahocorasick.git

In [2]:
import os
import json
import random
from matplotlib import pyplot as plt
import numpy as np
from tokenizer import tokenizer,token2str,vocab_size
import torch
import torch.nn as nn
from make_model import make_model
from train_and_use import Batch,CrossEntropyLoss,SimpleAdamOptimizer,OptimizerWrapper,train_server_start
from train_and_use import greedy_decode #使用text_continue时重复惩罚会严重减速，换用greedy_decode
from train_and_use import TOGGLE,STOP
from train_and_use import record
from tqdm import tqdm
import concurrent.futures
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
# %matplotlib widget

In [3]:
model = make_model(
    #token是从1开始的，0填充，剩下的用来覆盖全部字节
    vocab_size = vocab_size+1+255,
    embedding_dim = 16,
    key_dim = 8,
    head_number = 2,
    position_information_type = "mask",
    # position_information_type = "sinusoidal",
    # position_information_type = "rotary",
    # position_information_type = "learned",
    enable_affine = True,
    enable_talking_head = False,
    use_diff = False,
    self_attention_block_size = 0,
    feed_forward_dim = 16,
    enable_layer_norm = True,
    deep = 1,
    dropout_rate = 0.1
).to(device)

In [4]:
model.load_state_dict(torch.load('fastautotagger.weight',weights_only=True))
model = model.eval()

In [5]:
file_high = open('high_ff.txt','a')
file_mid = open('mid_ff.txt','a')
file_low  = open('low_ff.txt','a')

In [6]:
fnames = os.listdir('WuDaoCorpus2.0_base_200G')
fnames.sort()
fnames = fnames[67:]
cnt = 0
prompt = tokenizer(' 这段文本的质量按照“高、中、低”三档评价为：',5.0)
tokens_batch = []
data_ff = []
for fname in fnames:
    cnt += 1
    print('\r[',cnt,'/',len(fnames),']',end=' '*10)
    with open('WuDaoCorpus2.0_base_200G/'+fname,'r',encoding='utf-8') as f:
        lines = [item['content'] for item in json.load(f)]
        with torch.amp.autocast("cuda"):
            with concurrent.futures.ProcessPoolExecutor(max_workers=14) as executor:
                task_dict = {executor.submit(tokenizer,line,5):line for line in lines}
                with tqdm(total=len(task_dict)) as pbar:
                    for future in concurrent.futures.as_completed(task_dict):
                        data_ff += [task_dict[future]]
                        task_dict[future] = ""
                        tokens_batch += [(future.result()+[-255]*1000)[:1000] + prompt]
                        if len(tokens_batch) == 256:
                            tokens_batch = np.array(tokens_batch,dtype=np.int64)+255
                            inputs = torch.from_numpy(tokens_batch).to(device).data
                            with torch.no_grad():
                                o = greedy_decode(model,inputs,out_length=1)
                            for res,line in zip(o,data_ff):
                                res = token2str(res.cpu().numpy()-255)[-1]
                                if res == '高':
                                    print(line,file=file_high)
                                if res == '中':
                                    print(line,file=file_mid)
                                if res == '低':
                                    print(line,file=file_low)
                            tokens_batch = []
                            data_ff = []
                        pbar.update(1)
            if len(tokens_batch) != 0:
                tokens_batch = np.array(tokens_batch,dtype=np.int64)+255
                inputs = torch.from_numpy(tokens_batch).to(device).data
                with torch.no_grad():
                    o = greedy_decode(model,inputs,out_length=1)
                for res,line in zip(o,data_ff):
                    res = token2str(res.cpu().numpy()-255)[-1]
                    if res == '高':
                        print(line,file=file_high)
                    if res == '中':
                        print(line,file=file_mid)
                    if res == '低':
                        print(line,file=file_low)
                tokens_batch = []
                data_ff = []

[ 1 / 299 ]          

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 533162/533162 [13:20<00:00, 665.64it/s]


[ 2 / 299 ]          

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 393178/393178 [13:43<00:00, 477.57it/s]


[ 3 / 299 ]          

...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 49938/49938 [00:48<00:00, 1034.51it/s]


[ 298 / 299 ]          

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 48875/48875 [00:45<00:00, 1070.54it/s]


[ 299 / 299 ]          

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 63894/63894 [00:55<00:00, 1143.74it/s]


In [7]:
file_high.close()
file_mid.close()
file_low.close()